
## Projeto ICMC Júnior
### Objetivo: Prever se um funcionário vai sair da empresa (attrition).

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
# Leitura do arquivo csv
df = pd.read_csv('dados.csv')
# Verificação do tamanho da base de dados
df.shape

(1470, 35)

In [ ]:
def calculate_lim_iqr(data_column):
    """Função responsável por calcular a amplitude interquartil de um Série (Coluna)"""
    Q1 = data_column.quantile(0.25) # 1° Quartil 
    Q3 = data_column.quantile(0.75) # 3° Quartil 
    
    
    IQR = Q3 - Q1 # amplitude interquartil
    
    limite_inferior = Q1 - (1.5 * IQR) # lim inferior do box-plot
    limite_superior = Q3 + (1.5 * IQR) # lim superior do box-plot
    
    return limite_inferior, limite_superior

In [90]:
# Exibe os 5 primeiros funcionários da base de dados, apenas para contextualizar com a base
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,2,Life Sciences,1.0,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,1,Life Sciences,1.0,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2.0,2,Other,1.0,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,NaN,4,Life Sciences,NaN,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,NaN,591,Research & Development,2.0,1,Medical,NaN,7,...,4,80,1,6,3,3,2,2,2,2


In [91]:
# TRATAMENTO DE VALORES NULOS
df.isnull().sum() # Verifica quantos valores nulos tem por colunas, ou seja, por categoria.

Age                           0
Attrition                     0
BusinessTravel               70
DailyRate                     0
Department                    0
DistanceFromHome             47
Education                     0
EducationField                0
EmployeeCount               220
EmployeeNumber                0
EnvironmentSatisfaction      53
Gender                       67
HourlyRate                   32
JobInvolvement               61
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                     51
PercentSalaryHike             0
PerformanceRating            46
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [92]:
df['EmployeeCount']

0       1.0
1       1.0
2       1.0
3       NaN
4       NaN
       ... 
1465    1.0
1466    1.0
1467    1.0
1468    1.0
1469    1.0
Name: EmployeeCount, Length: 1470, dtype: float64

In [93]:
# Em 'EmployeeCount' todos os valores são como padrão igual a 1.0, sendo cada linha, um funcionário, com isso se corrige os NaN para 1.0
# no dataframe original
df['EmployeeCount'].fillna(value=1, inplace=True)

In [ ]:
# Tratamento dos valores nulos da coluna: DistanceFromHome (Quantitativa continua)
# Calculando o IQR = Q3 - Q1

# Calcula o limite inferior e limite superior da coluna DistanceFromHome
lim_inf_dfh,lim_sup_dfh = calculate_lim_iqr(df['DistanceFromHome'])

print(f"Limite Inferior (Q1 - 1.5*IQR): {lim_inf_dfh}")
print(f"Limite Superior (Q3 + 1.5*IQR): {lim_sup_dfh}")

Limite Inferior (Q1 - 1.5*IQR): -16.0
Limite Superior (Q3 + 1.5*IQR): 32.0


In [95]:
# Resumo das descrições estatísticas da categoria DistanceFromHome
df['DistanceFromHome'].describe()

count    1423.000000
mean        9.188335
std         8.133707
min         1.000000
25%         2.000000
50%         7.000000
75%        14.000000
max        29.000000
Name: DistanceFromHome, dtype: float64

In [96]:
# O menor valor (min) é igual a 1.0 e o maior valor (max) é igual a 29.0, com isso podemos ver atráves disso, todos 
# os dados então dentro das margens, não tendo nenhum outliers. No entanto, devido ao desvio padrão ser mais de 80% da média, 
# indica que meus dados, não estão bem distribuidos. 

# Calculo a mediana
median_distances = df['DistanceFromHome'].median() 
# Preenche todos os valores NaN com a mediana e em seguida, reatribui a coluna nela mesma.
df['DistanceFromHome'] = df['DistanceFromHome'].fillna(median_distances)


In [97]:
print(df['EnvironmentSatisfaction'].value_counts(dropna=False))

EnvironmentSatisfaction
3.0    438
4.0    427
2.0    277
1.0    275
NaN     53
Name: count, dtype: int64


In [98]:
# Tratamento da coluna EnvironmentSatisfaction - Qualitativa ordinal
# Preenche os valores ausentes com os valores da moda, em seguida, reatribui os valores na coluna original do dataframe
# Obs: a mediana aqui teria o mesmo resultado da moda.

# Pega o primeiro (e único) valor da moda
moda_env_satisfaction = df['EnvironmentSatisfaction'].mode()[0]
df['EnvironmentSatisfaction'] = df['EnvironmentSatisfaction'].fillna(moda_env_satisfaction)

In [99]:
print(df['OverTime'].value_counts(dropna=False, normalize=True)) # Exibe a proporção de cada resposta na coluna OverTime, variando de 0 a 1.

OverTime
No     0.693197
Yes    0.272109
NaN    0.034694
Name: proportion, dtype: float64


In [100]:
# Tratamento de valores ausentes coluna - OverTime (Qualitativo nominal)
# Como podemos ver acima, possui uma quantidade esmagadora de funcionários que não realizam horas extras, quase 70% deles.

# Preenche os valores ausentes com a moda e reatribui os resultados na coluna respectiva
df['OverTime'] = df['OverTime'].fillna(df['OverTime'].mode()[0])


In [104]:
print(df['HourlyRate'].describe())

count    1438.000000
mean       65.907510
std        20.386193
min        30.000000
25%        48.000000
50%        66.000000
75%        84.000000
max       100.000000
Name: HourlyRate, dtype: float64


In [112]:
# Tratamento valores ausentes - HourlyRate (Quantitativo Contínua)

# Calcula o limite inferior e limite superior da coluna HourlyRate
lim_inf_hr, lim_sup_hr = calculate_lim_iqr(df['HourlyRate']) 
print(f"Limite inferior:{lim_inf_hr}, limite superior: {lim_sup_hr}")
print(f"Coeficiente de variação {df['HourlyRate'].std() / df['HourlyRate'].mean()}")
print(f"Média: { df['HourlyRate'].mean()} e Mediana: { df['HourlyRate'].median()}")

Limite inferior:-6.0, limite superior: 138.0
Coeficiente de variação 0.3093151741293963
Média: 65.90751043115438 e Mediana: 66.0


In [ ]:
# Podemos ver atráves dos limites (inferior e superior) que os nossos dados estão dentro dessas margens, visto que
# o menor valor (min) é 30.0 e o maior (max) é 100.0, com isso não possui outliers. O coeficiente de variação é igual a 30%, dessa forma
# temos uma variação média entre os dados. Além disso, a média está bem proxima da mediana. Usa-se a média para representação dos dados ausentes

# Preenche os valores ausentes com a média e reatribui os resultados na coluna respectiva
df['HourlyRate'] = df['HourlyRate'].fillna(df['HourlyRate'].mean())

In [117]:
df.isnull().sum() # Verifica quantos valores nulos tem por colunas, ou seja, por categoria.

Age                          0
Attrition                    0
BusinessTravel              70
DailyRate                    0
Department                   0
DistanceFromHome             0
Education                    0
EducationField               0
EmployeeCount                0
EmployeeNumber               0
EnvironmentSatisfaction      0
Gender                      67
HourlyRate                   0
JobInvolvement              61
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                0
MonthlyIncome                0
MonthlyRate                  0
NumCompaniesWorked           0
Over18                       0
OverTime                     0
PercentSalaryHike            0
PerformanceRating           46
RelationshipSatisfaction     0
StandardHours                0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInC